# Custom Language Model Code Generation Testing

This notebook demonstrates the capabilities of a custom-trained language model ('MyOwnLLM') for generating Python code snippets. The model is based on the transformers library and is tested on various coding tasks to evaluate its code generation abilities.

## Setup and Dependencies
We'll use the transformers library to load and run our model, along with some utility functions for processing the generated code.

In [7]:
# Import required libraries and initialize the model pipeline
from transformers import pipeline, set_seed

# Load our custom trained model
model_ckpt = 'MyOwnLLM'
generation = pipeline('text-generation', model=model_ckpt)

## Utility Functions

Below we define helper functions to:
1. Extract the first code block from generated text
2. Generate multiple code completions with controlled randomness

In [8]:
import re
from transformers import set_seed 

def first_block(string):
    """Extract the first code block before any new class/function definitions or special tokens."""
    return re.split('\nclass|\ndef|\n#|\n@|\nprint|\nif', string)[0].rstrip()

def complete_code(pipe, prompt, max_length=64, num_completions=4, seed=1):
    """Generate multiple code completions for a given prompt.
    
    Args:
        pipe: The transformer pipeline
        prompt: Input code prompt
        max_length: Maximum length of generated text
        num_completions: Number of different completions to generate
        seed: Random seed for reproducibility
    """
    set_seed(seed)
    # Set generation parameters for controlled randomness
    gen_kwargs = {
        "temperature": 0.4,  # Lower temperature for more focused sampling
        "top_p": 0.95,      # Nucleus sampling parameter
        "top_k": 0,         # Disabled top-k sampling
        "num_beams": 1,     # Simple sampling without beam search
        "do_sample": True,
    }
    code_gens = generation(prompt, num_return_sequences=num_completions, 
                            max_length=max_length, **gen_kwargs)
    code_strings = []
    for code_gen in code_gens:
        generated_code = first_block(code_gen['generated_text'][len(prompt):])
        code_strings.append(generated_code)
    print(('\n'+'='*80 + '\n').join(code_strings))

## Test Case 1: Simple Mathematical Function

Testing the model's ability to complete a basic geometric function for calculating rectangle area.

In [9]:
# Test prompt for rectangle area calculation
prompt = '''def area_of_rectangle(a: float, b: float):
    """Return the area of the rectangle."""'''
complete_code(generation, prompt)

# Note: The model generates multiple solutions, but only the first one (a * b)
# would be mathematically correct for rectangle area

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



    return (a - b) * (a - b)

    return a * b / (a + b)

    return (a + b / 2.0) * area_of_rectangle(a, b)

    return (a - b) * area_of_rectangle(a, b)


## Test Case 2: URL Extraction from HTML

Testing the model's ability to generate more complex code involving string processing and regular expressions.

In [10]:
# Test prompt for HTML URL extraction
prompt = '''def get_urls_from_html(html):
    """Get all embedded URLs in a HTML string."""'''
complete_code(generation, prompt)

# Note: The last generated solution using re.finditer is the most practical approach

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



    urls = []
    for match in re.finditer(r'(<[^>]*\bhref=["\']?[^>]*\bstyle=["\']?([^>]*)', html):
        urls.append(match.

    if not html:
        return ''
    if len(html) == 1:
        return get_urls_from_html(html[0])
    else:
        return get_urls_from_

    if not html:
        return ''
    if not html.startswith('<script'):
        return ''
    return html.split('<script ')[1]

    urls = []
    for match in re.finditer('<a href="(.*?)">', html):
        urls.append(match.group(1))
    return urls


## Test Case 3: Function Implementation Comparison

Testing the model's ability to translate between native Python and NumPy implementations.

In [11]:
# Test prompt comparing native Python vs NumPy implementation
prompt = '''# a function in native python:
def mean(a):
    return sum(a)/len(a)

# the same function using numpy:
import numpy as np
def mean(a):'''
complete_code(generation, prompt, max_length=64)

# Note: The model correctly identifies np.mean as the NumPy equivalent

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



    return sum(a)/len(a)

    return np.mean(a)

    return np.mean(a)

    return sum(a)/len(a)


## Test Case 4: Machine Learning Code Generation

Testing the model's ability to generate machine learning setup code using scikit-learn.

In [13]:
# Test prompt for random forest classifier setup
prompt = '''X = np.random.randn(100, 100)
y = np.random.randint(0, 1, 100)

# fit random forest classifier with 20 estimators'''
complete_code(generation, prompt, max_length=156)

# Note: The model consistently generates correct RandomForestClassifier initialization and fitting

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



clf = RandomForestClassifier(n_estimators=20, random_state=1)
clf.fit(X, y)

clf = RandomForestClassifier(n_estimators=20)
clf.fit(X, y)

regr_1 = DecisionTreeRegressor(max_depth=4)

regr_2 = AdaBoostRegressor(DecisionTreeRegressor(max_depth=4),
                          n_estimators=300, random_state=0)

regr_1.fit(X, y)
regr_2.fit(X, y)

clf = RandomForestClassifier(n_estimators=20, random_state=1)
clf.fit(X, y)
